In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [15]:
path = "C:\\Users\\Republic Of Gamers\\OneDrive\\Documents\\GitHub\\TSDN-BoyWithLuv\\Source\\Data\\sdm_emergency_monthly.csv"
path2 = "C:\\Users\\Republic Of Gamers\\OneDrive\\Documents\\GitHub\\TSDN-BoyWithLuv\\Source\\Data\\sdm_emergency_monthly_gender_only.csv"
path3 = "C:\\Users\\Republic Of Gamers\\OneDrive\\Documents\\GitHub\\TSDN-BoyWithLuv\\Source\\Data\\sdm_emergency_monthly_gender_RhFactor.csv"
emergency_df = pd.read_csv(path)
emergency_df.head()

,Emergency,"('Emergency', 'A')","('Emergency', 'AB')","('Emergency', 'B')","('Emergency', 'O')","('Emergency', 'Female')","('Emergency', 'Male')",Emergency_Lag_1,Emergency_Lag_2,Emergency_Lag_3
0,305,81,77,72,75,157,148,308.0,275.0,224.0
1,300,73,71,83,73,140,160,305.0,308.0,275.0
2,306,79,77,78,72,174,132,300.0,305.0,308.0
3,284,73,60,75,76,144,140,306.0,300.0,305.0
4,308,79,72,87,70,168,140,284.0,306.0,300.0


In [4]:
emergency_df_gender = pd.read_csv(path2)
emergency_df_gender.head()

,Emergency,"('Emergency', 'Female')","('Emergency', 'Male')",Emergency_Lag_1,Emergency_Lag_2,Emergency_Lag_3
0,305,157,148,308.0,275.0,224.0
1,300,140,160,305.0,308.0,275.0
2,306,174,132,300.0,305.0,308.0
3,284,144,140,306.0,300.0,305.0
4,308,168,140,284.0,306.0,300.0


In [16]:
emergency_df_gender_rhfactor = pd.read_csv(path3)
emergency_df_gender_rhfactor.head()

,Emergency,"('Emergency', '+')","('Emergency', '-')","('Emergency', 'Female')","('Emergency', 'Male')",Emergency_Lag_1,Emergency_Lag_2,Emergency_Lag_3
0,305,147,158,157,148,308.0,275.0,224.0
1,300,151,149,140,160,305.0,308.0,275.0
2,306,159,147,174,132,300.0,305.0,308.0
3,284,144,140,144,140,306.0,300.0,305.0
4,308,151,157,168,140,284.0,306.0,300.0


In [5]:
emergency_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Emergency                58 non-null     int64  
 1   ('Emergency', 'A')       58 non-null     int64  
 2   ('Emergency', 'AB')      58 non-null     int64  
 3   ('Emergency', 'B')       58 non-null     int64  
 4   ('Emergency', 'O')       58 non-null     int64  
 5   ('Emergency', 'Female')  58 non-null     int64  
 6   ('Emergency', 'Male')    58 non-null     int64  
 7   Emergency_Lag_1          58 non-null     float64
 8   Emergency_Lag_2          58 non-null     float64
 9   Emergency_Lag_3          58 non-null     float64
dtypes: float64(3), int64(7)
memory usage: 4.7 KB


In [5]:
emergency_df_gender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Emergency                58 non-null     int64  
 1   ('Emergency', 'Female')  58 non-null     int64  
 2   ('Emergency', 'Male')    58 non-null     int64  
 3   Emergency_Lag_1          58 non-null     float64
 4   Emergency_Lag_2          58 non-null     float64
 5   Emergency_Lag_3          58 non-null     float64
dtypes: float64(3), int64(3)
memory usage: 2.8 KB


In [17]:
emergency_df_gender_rhfactor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Emergency                58 non-null     int64  
 1   ('Emergency', '+')       58 non-null     int64  
 2   ('Emergency', '-')       58 non-null     int64  
 3   ('Emergency', 'Female')  58 non-null     int64  
 4   ('Emergency', 'Male')    58 non-null     int64  
 5   Emergency_Lag_1          58 non-null     float64
 6   Emergency_Lag_2          58 non-null     float64
 7   Emergency_Lag_3          58 non-null     float64
dtypes: float64(3), int64(5)
memory usage: 3.8 KB


In [6]:
trainEmergencySize = int(len(emergency_df) * 0.7)
trainEmergency, testEmergency= emergency_df[:trainEmergencySize], emergency_df[trainEmergencySize:]

In [6]:
trainEmergencySizeGender = int(len(emergency_df_gender) * 0.7)
trainEmergencyGender, testEmergencyGender= emergency_df_gender[:trainEmergencySizeGender], emergency_df_gender[trainEmergencySizeGender:]

In [18]:
trainEmergencySizeGenderRhFactor = int(len(emergency_df_gender_rhfactor) * 0.7)
trainEmergencyGenderRhFactor, testEmergencyGenderRhFactor= emergency_df_gender_rhfactor[:trainEmergencySizeGenderRhFactor], emergency_df_gender_rhfactor[trainEmergencySizeGenderRhFactor:]

In [8]:
X_train = trainEmergency.drop(columns=['Emergency'])
y_train = trainEmergency['Emergency']
X_test = testEmergency.drop(columns=['Emergency'])
y_test = testEmergency['Emergency']
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [7]:
X_train = trainEmergencyGender.drop(columns=['Emergency'])
y_train = trainEmergencyGender['Emergency']
X_test = testEmergencyGender.drop(columns=['Emergency'])
y_test = testEmergencyGender['Emergency']
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [21]:
X_train = trainEmergencyGenderRhFactor.drop(columns=['Emergency'])
y_train = trainEmergencyGenderRhFactor['Emergency']
X_test = testEmergencyGenderRhFactor.drop(columns=['Emergency'])
y_test = testEmergencyGenderRhFactor['Emergency']
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [8]:
# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize the model
rf = RandomForestRegressor()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit to the data
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", grid_search.best_params_)


Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters found:  {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}


In [9]:
# Extract the best parameters from the grid search
best_params = grid_search.best_params_

# Initialize the RandomForestRegressor with the best parameters
best_rf = RandomForestRegressor(**best_params)

# Fit the model to the full training data
best_rf.fit(X_train, y_train)

# Make predictions on the test set or future data
y_pred = best_rf.predict(X_test)

# Display predictions
print(y_pred)

[297.29 314.36 295.34 322.63 304.88 312.97 309.66 292.14 309.27 305.78
 307.52 300.73 311.33 289.92 304.3  296.51 321.01 274.44]


In [10]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

# Display the metrics
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R²):", r2)

Root Mean Squared Error (RMSE): 49.83283188153502
Mean Absolute Error (MAE): 15.442222222222215
R-squared (R²): 0.22258860482103715


RhFactor and Gender RF

In [22]:
#RhFactor and Ge
# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize the model
rf = RandomForestRegressor()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit to the data
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", grid_search.best_params_)


Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters found:  {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}


In [23]:
# Extract the best parameters from the grid search
best_params = grid_search.best_params_

# Initialize the RandomForestRegressor with the best parameters
best_rf = RandomForestRegressor(**best_params)

# Fit the model to the full training data
best_rf.fit(X_train, y_train)

# Make predictions on the test set or future data
y_pred = best_rf.predict(X_test)

# Display predictions
print(y_pred)

[296.52333333 315.66333333 294.86333333 314.94833333 304.94333333
 313.45833333 310.54333333 296.5        308.20833333 302.58
 308.93166667 296.11       308.24166667 290.27       303.97166667
 295.27       323.45       276.19      ]


In [24]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

# Display the metrics
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R²):", r2)

Root Mean Squared Error (RMSE): 50.25863878509708
Mean Absolute Error (MAE): 15.482777777777772
R-squared (R²): 0.20924634064566483


In [11]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
xgb_model = XGBRegressor()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit to the data
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", grid_search.best_params_)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters found:  {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}


c:\Users\Republic Of Gamers\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:160: UserWarning: [22:24:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)


In [12]:
# Extract the best parameters from the grid search
best_params = grid_search.best_params_

# Initialize the XGBRegressor with the best parameters
best_xgb = XGBRegressor(**best_params)

# Fit the model to the full training data
best_xgb.fit(X_train, y_train)

# Make predictions on the test set or future data
y_pred = best_xgb.predict(X_test)

# Display predictions
print("Predictions:", y_pred)

Predictions: [295.4704  316.57498 294.72906 307.71524 304.33252 328.97305 312.1603
 291.84128 310.16037 303.05353 310.93164 299.4369  308.61447 290.1091
 307.13437 297.8799  328.9711  270.00443]


c:\Users\Republic Of Gamers\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:160: UserWarning: [22:24:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)


In [13]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

# Display the metrics
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R²):", r2)

Root Mean Squared Error (RMSE): 48.935411262582285
Mean Absolute Error (MAE): 15.685889350043404
R-squared (R²): 0.2503367066383362


RhFactor and Gender

In [25]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
xgb_model = XGBRegressor()

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit to the data
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters found: ", grid_search.best_params_)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters found:  {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}


c:\Users\Republic Of Gamers\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:160: UserWarning: [23:04:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)


In [26]:
# Extract the best parameters from the grid search
best_params = grid_search.best_params_

# Initialize the XGBRegressor with the best parameters
best_xgb = XGBRegressor(**best_params)

# Fit the model to the full training data
best_xgb.fit(X_train, y_train)

# Make predictions on the test set or future data
y_pred = best_xgb.predict(X_test)

# Display predictions
print("Predictions:", y_pred)

Predictions: [293.51028 311.71014 291.4862  306.1702  300.69055 324.62357 312.84024
 296.64117 309.89227 304.76337 310.27762 292.06677 311.92944 289.99612
 307.31836 298.46628 335.54422 271.13834]


c:\Users\Republic Of Gamers\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:160: UserWarning: [23:04:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)


In [27]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

# Display the metrics
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R²):", r2)

Root Mean Squared Error (RMSE): 49.335912471370925
Mean Absolute Error (MAE): 16.731548733181423
R-squared (R²): 0.23801559209823608
